In [36]:
import cv2 as cv
import numpy as np
from PIL import Image

import pytesseract

main_image='ingre.jpg'

# Set the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

def extractTextFromImage(image):
    custom_config = r'--oem 3 -l eng'
    extractedText = pytesseract.image_to_string(image,config=custom_config)
    return extractedText
input_image_path = main_image
extracted_text = extractTextFromImage(input_image_path)

# Now you can work with the extracted text
print(extracted_text)

LOW FAT VANILLA FLAVOURED YOGHURT

INGREDIENTS: Skim Milk, Concentrated
Skim Milk, Water, Sugar, Cream (From

Milk), Thickeners (1422 (From Maize),

1442 (From Maize)), Milk Solids,

Gelatine, Flavours, Acidity Regulators
(331, 332, 270, 330), Enzyme
(Lactase), Live Cultures.

Contains Milk and Milk Products.




In [37]:
import cv2 as cv
import numpy as np
from PIL import Image

def preprocess_image(image_path, output_path):
    # Load the image using OpenCV
    img = cv.imread(image_path)

    # Check if the image is loaded successfully
    if img is not None:
        # Image scaling using PIL
        im = Image.open(image_path)
        im.save("test-300.png", dpi=(300, 300))

        # Image contrast and brightness adjustment
        alpha = 1.5  # Contrast control (1.0-3.0)
        beta = 0  # Brightness control (0-100)
        adjusted = cv.convertScaleAbs(img, alpha=alpha, beta=beta)

        # Binarize the image (you can use 'im' if it's binarized in the previous step)
        im_gray = cv.imread("test-300.png", cv.IMREAD_GRAYSCALE)
        (thresh, im_bw) = cv.threshold(im_gray, 128, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
        im_bw = cv.threshold(im_gray, thresh, 255, cv.THRESH_BINARY)[1]

        # Skew correction (your existing function)
        def deskew(image):
            # Ensure that the input image is not empty
            if image is None or np.count_nonzero(image) == 0:
                return image

            # Apply thresholding to create a binary image
            _, binary_image = cv.threshold(image, 128, 255, cv.THRESH_BINARY)

            # Find non-zero pixel coordinates
            coords = np.column_stack(np.where(binary_image > 0))

            # Ensure there are enough coordinates to calculate the minimum area rectangle
            if len(coords) < 2:
                return image

            # Calculate the minimum area rectangle
            angle = cv.minAreaRect(coords)[-1]

            if angle < -45:
                angle = -(90 + angle)
            else:
                angle = -(angle)+90

            (h, w) = image.shape[:2]
            center = (w // 2, h // 2)
            M = cv.getRotationMatrix2D(center, angle, 1.0)
            rotated = cv.warpAffine(image, M, (w, h), flags=cv.INTER_CUBIC, borderMode=cv.BORDER_REPLICATE)
            return rotated

        # Apply skew correction to the binarized image
        rotated_image = deskew(im_bw)

        # Save the preprocessed image as a new file
        cv.imwrite(output_path, rotated_image)

# Provide the path to your input image and output image
input_image_path = main_image
output_image_path = 'preprocessed_image.png'

# Call the preprocess_image function to preprocess and save the image
preprocess_image(input_image_path, output_image_path)

# Check if the preprocessing and saving were successful
print(f"Preprocessed image saved as {output_image_path}")


Preprocessed image saved as preprocessed_image.png


In [38]:
from PIL import Image, ImageOps
import os

image_path = r"preprocessed_image.png"

def compress_image(input_path, target_size):
    image = Image.open(input_path)
    image = ImageOps.exif_transpose(image)  # Correct the image orientation
   
    quality = 100
    while os.path.exists(input_path) and os.path.getsize(input_path) > target_size:
        print("Compressing Image")
        print(os.path.getsize(input_path))
        # Save the image with the current quality level
        image.save(input_path, optimize=True, quality=quality)

        # Reduce the quality level by 10 units
        quality -= 10

        # Break the loop if the quality level reaches 0
        if quality <= 0:
            break
    # Optionally, you can reapply the EXIF transpose if needed
    image = ImageOps.exif_transpose(image)
    
    # Return the compressed image
    return image

# Define the target size you want and call the function with it
target_size = 1024 * 1024  # For example, 1 MB
compressed_image = compress_image(image_path, target_size)

# You can now use the 'compressed_image' for further processing or saving
if compressed_image is not None:
    # You can use 'compressed_image' for further processing or display
    compressed_image.show()
else:
    print(f"Error: Unable to load the image at '{image_path}'")

In [39]:
from PIL import Image, ImageOps
import os


image_path = r"preprocessed_image.png"

def compress_image(input_path, target_size):
    image = Image.open(input_path)
    image = ImageOps.exif_transpose(image)  # Correct the image orientation
   
    quality = 100
    while os.path.exists(input_path) and os.path.getsize(input_path) > target_size:
        print("Compressing Image")
        print(os.path.getsize(input_path))
        # Save the image with the current quality level
        image.save(input_path, optimize=True, quality=quality)

        # Reduce the quality level by 10 units
        quality -= 10

        # Break the loop if the quality level reaches 0
        if quality <= 0:
            break
    # Optionally, you can reapply the EXIF transpose if needed
    image = ImageOps.exif_transpose(image)
    
    # Return the compressed image
    return image

# Define the target size you want and call the function with it
target_size = 1024 * 1024  # For example, 1 MB
compressed_image = compress_image(image_path, target_size)

# You can now use the 'compressed_image' for further processing or saving
if compressed_image is not None:
    # You can use 'compressed_image' for further processing or display
    compressed_image.show()
else:
    print(f"Error: Unable to load the image at '{image_path}'")


In [40]:
def extractTextFromImage(image):
    custom_config = r'--oem 3 -l eng'
    extractedText = pytesseract.image_to_string(image,config=custom_config)
    return extractedText

extracted_text = extractTextFromImage(compressed_image)

# Now you can work with the extracted text
print(extracted_text)

LOW FAT VANILLA FLAVOURED YOGHURT

INGREDIENTS: Skim Milk, Concentrated
Skim Milk, Water, Sugar, Cream (From

Milk), Thickeners (1422 (From Maize),

1442 (From Maize)), Milk Solids,

Gelatine, Flavours, Acidity Regulators
(331, 332, 270, 330), Enzyme
(Lactase), Live Cultures.

Contains Milk and Milk Products.




In [41]:
def extractTextFromImage(image):
    custom_config = r'--oem 3 -l eng'
    extractedText = pytesseract.image_to_string(image,config=custom_config)
    return extractedText
input_image_path = main_image
extracted_text = extractTextFromImage(input_image_path)

# Now you can work with the extracted text
print(extracted_text)

LOW FAT VANILLA FLAVOURED YOGHURT

INGREDIENTS: Skim Milk, Concentrated
Skim Milk, Water, Sugar, Cream (From

Milk), Thickeners (1422 (From Maize),

1442 (From Maize)), Milk Solids,

Gelatine, Flavours, Acidity Regulators
(331, 332, 270, 330), Enzyme
(Lactase), Live Cultures.

Contains Milk and Milk Products.




In [42]:
with open("OCR_TEXT.txt","w") as fp:
    fp.write(extracted_text)